In [1]:
import payment_simulator as ps
from payment_simulator.anomaly import AnomalyGenerator
from payment_simulator.networks import SimplePaymentNetwork
from payment_simulator.utils import random_payment_timing, random_payment_value

In [2]:
sim_periods = list(range(15))

sim_params = {
    "open_time": "06:30:00",
    "close_time": "18:30:00",
    "value_fn": random_payment_value,
    "timing_fn": random_payment_timing,
}

payment_network = SimplePaymentNetwork(
    total_banks=10, avg_payments=15, alpha=0.01, allow_self_loop=False
)

anomaly_generator = AnomalyGenerator(
    anomaly_start=5,
    anomaly_end=10,
    prob_start=0.8,
    prob_end=1,
    lambda_start=0.5,
    lambda_end=2.5,
    rate=0.5,
)

In [3]:
normal_transactions = ps.TransactionSim(
    sim_id=1, network=payment_network, **sim_params
)
normal_transactions.run(sim_periods)

payments1 = normal_transactions.get_payments_df()
print(payments1.head(3))
print(payments1.tail(3))

   Period      Time  Sender  Receiver  Count     Value
0       0  18:02:13       0         3      1  0.593124
1       0  12:13:18       0         3      1  0.866371
2       0  11:01:37       0         3      1  1.257686
      Period      Time  Sender  Receiver  Count     Value
2247      14  16:01:40       8         5      1  1.136979
2248      14  07:55:55       8         1      1  2.230251
2249      14  12:02:00       9         7      1  7.638875


In [4]:
anomaly_transactions = ps.AnomalyTransactionSim(
    sim_id=2, network=payment_network, anomaly=anomaly_generator, **sim_params
)
anomaly_transactions.run(sim_periods)

payments2 = anomaly_transactions.get_payments_df()
print(payments2.head(3))
print(payments2.tail(3))

   Period      Time  Sender  Receiver  Count     Value
0       0  15:24:40       0         1      1  0.573187
1       0  07:51:37       0         1      1  1.094547
2       0  12:58:54       0         1      1  1.451194
      Period      Time  Sender  Receiver  Count     Value
2247      14  14:44:05       9         5      1  3.670616
2248      14  15:55:43       9         6      1  1.563775
2249      14  08:21:50       9         1      1  2.700776


In [5]:
print(f"Total Transaction of Normal RTGS  : {payments1['Value'].sum():.3f}")
print(f"Total Transaction of Anomaly RTGS : {payments2['Value'].sum():.3f}")

Total Transaction of Normal RTGS  : 3624.261
Total Transaction of Anomaly RTGS : 5198.998


In [6]:
correct = 0
test_len = 50

for _ in range(test_len):
    normal_transactions.run(sim_periods)
    anomaly_transactions.run(sim_periods)
    x1 = normal_transactions.get_payments_df()["Value"].sum()
    x2 = anomaly_transactions.get_payments_df()["Value"].sum()
    if x2 > x1:
        correct += 1

print(f"Success rate: {correct / test_len * 100:.2f}%")

Success rate: 100.00%
